# I/O using `HERAData` and `HERACal`

by Josh Dillon (jsdillon@berkeley.edu)

In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import os
from hera_cal.io import HERAData, HERACal
from hera_cal.data import DATA_PATH
uvh5_file_1 = os.path.join(DATA_PATH, "zen.2458116.61019.xx.HH.h5XRS_downselected")
uvh5_file_2 = os.path.join(DATA_PATH, "zen.2458116.61765.xx.HH.h5XRS_downselected")
miriad_file = os.path.join(DATA_PATH, "zen.2458043.12552.xx.HH.uvORA")
calfits_file = os.path.join(DATA_PATH, "test_input/zen.2457698.40355.xx.HH.uvc.omni.calfits")

## Reading and writing data with `HERAData`

`HERAData` is a subclass of `pyuvdata.UVData` with streamlined interfaces and additional functionality for producing and ingesting `DataContainer`s, the standard in-memory format for `hera_cal`. The standard syntax for loading an entire file is:

In [2]:
hd = HERAData(uvh5_file_1)
data, flags, nsamples = hd.read()

`data`, `flags`, and `nsamples` are `DataContainer`s and they include ***copies*** of data in the HERAData object. They work like dictionaries, mapping 3-tuples of the form `(0, 1, 'xx')` to waterfalls of shape `(Nint, Nfreq)`. They know to conjugate data when you ask for the reverse baseline are agnostic as to the capitalization convention for the polarization string. 

In [3]:
print(data.keys())
print(data[53, 54, 'xx'].shape)
print(data[53, 54, 'xx'][0,0])
print(data[54, 53, 'XX'][0,0])
print(data[54, 53, 'xx'][0,0])

[(53, 53, 'xx'), (53, 54, 'xx'), (54, 54, 'xx')]
(60, 1024)
(-16.241686+56.264435j)
(-16.241686-56.264435j)
(-16.241686-56.264435j)


A new feature is that the `DataContainer`s now include a bunch of useful metadata as class attributes. This includes:
* `data.antpos`: a dictionary that maps antenna number to position in meters
* `data.freqs`: a numpy array of the frequencies in Hz
* `data.times`: a numpy array of the unique times in the data in JD
* `data.lsts`: a numpy array of the unique LSTs in the data in radians
* `data.times_by_bl`: a dictionary that maps antenna pairs to JDs (will be useful for baseline-dependant averaging)
* `data.lsts_by_bl`: a dictionary that maps antenna pairs to LSTS (will be useful for baseline-dependant averaging)

`HERAData`'s preferred (and defualt) format is `uvh5`, which is based on HDF5. When using this format, useful metadata about the whole file is stored `HERAData` object and it does not change when a partial data read is performed. `HERAData` also supports loading `miriad` and `uvfits`, but they do not get this useful whole-file metadata saved internally.

In [4]:
hd = HERAData(uvh5_file_1)
print('useful metadata:', hd.HERAData_metas)
print('uvh5 hd.freqs:', hd.freqs)

hd = HERAData(miriad_file, filetype='miriad')
print('miriad hd.freqs:', hd.freqs)
data, flags, nsamples = hd.read()

useful metadata: ['ants', 'antpos', 'freqs', 'times', 'lsts', 'pols', 'antpairs', 'bls', 'times_by_bl', 'lsts_by_bl']
uvh5 hd.freqs: [1.00000000e+08 1.00097656e+08 1.00195312e+08 ... 1.99707031e+08
 1.99804688e+08 1.99902344e+08]
miriad hd.freqs: None


`HERAData` also supports loading of lists of files:

In [5]:
hd = HERAData([uvh5_file_1, uvh5_file_2])
data, flags, nsamples = hd.read()
print(data[53, 54, 'xx'].shape)

(120, 1024)


### Going from DataContainers back to HERAData

`HERAData.read()` produces copies of the data in `DataContainer`s. To put data back in the internal data arrays, e.g. in preparation for writing out a file, use `update()`. Then once can use standard `UVData` write functionality, like `write_uvh5()`:

In [6]:
hd = HERAData(uvh5_file_1)
data, flags, nsamples = hd.read()
# do stuff here
hd.update(data=data, flags=flags, nsamples=nsamples)
hd.write_uvh5('new_file.uvh5', clobber=True)
os.remove('new_file.uvh5')

### Partial Data Loading

`HERAData` is designed with partial data loading in mind. It supports partial data loading by baseline, polarization, time, frequency, and channel (though the last three don't work as well in miriad).

In [7]:
hd = HERAData(uvh5_file_1)
data, flags, nsamples = hd.read(bls=[(53, 54, 'xx'), (54, 54, 'xx')], frequencies=hd.freqs[100:200], times=hd.times[0:20])
print('number of baselines:', len(data))
print('waterfall shape:', data[(53, 54, 'xx')].shape)

number of baselines: 2
waterfall shape: (20, 100)


Partial data reads replace rather than add to the data stored in the HERAData object.

### Iterators

This sort of partial data loading is made easier by useful methods for looping over an entire datafile. For example, if we want to iterate over frequencies 300 at a time, we could do the following:

In [8]:
hd = HERAData(uvh5_file_1)
for data, flags, nsamples in hd.iterate_over_freqs(Nchans=300):
    # do stuff here
    print('Partial load shape:', data[hd.bls[0]].shape)

Partial load shape: (60, 300)
Partial load shape: (60, 300)
Partial load shape: (60, 300)
Partial load shape: (60, 124)


This funcationality, and the analogous `iterate_over_bls()` and `iterate_over_times()` are by default only available for `uvh5`-initialized `HERAData` objects because only they simultaneously know the dimensions of the full file and of the current subset of the file loaded into memory. However it is possible to manually provide the list to iterator over when using other formats. For example:

In [9]:
hd = HERAData(miriad_file, filetype='miriad')
for data, flags, nsamples in hd.iterate_over_bls(Nbls=2, bls=[(52, 52, 'xx'), (52, 53, 'xx'), (53, 53, 'xx')]):
    # do stuff here
    print('Baselines loaded:', data.keys())

Baselines loaded: [(52, 52, 'xx'), (52, 53, 'xx')]
Baselines loaded: [(53, 53, 'xx')]


### Partial Data Writing

`uvh5` files now support partial data writing. As its enabled by HERAData, this produces an empty data file the same size as the input `uvh5` file and then writes a part of it corresponding to the last partial `read()` (iterators also call `read()` at each iterateration). The part of the data that is written will thus have the same dimensions as the data read most recently. So, for example:

In [10]:
hd = HERAData(uvh5_file_1)
for data, flags, nsamples in hd.iterate_over_freqs(Nchans=300):
    # do stuff here
    hd.partial_write('new_file.uvh5', data=data, flags=flags, nsamples=nsamples, clobber=True)
    print('Now writing', data.freqs[0]/1e6, 'to', data.freqs[-1]/1e6, 'MHz')
os.remove('new_file.uvh5')

Now writing 100.0 to 129.19921875 MHz
Now writing 129.296875 to 158.49609375 MHz
Now writing 158.59375 to 187.79296875 MHz
Now writing 187.890625 to 199.90234375 MHz


## Reading and writing calibration solutions with `HERACal`

Analogous to the new `HERAData` module, we've also subclassed `pyuvdata.UVCal` into `HERACal` which operates in a similar way, though it is far less fully featured.

In [11]:
hc = HERACal(calfits_file)
gains, flags, quals, total_qual = hc.read()

`HERACal`'s read produces dictionaries that map antenna-pol tuples to gain waterfalls of N 

Theses are:
* `gains`: dict mapping antenna-pol keys to (Nint, Nfreq) complex gains arrays
* `flags`: dict mapping antenna-pol keys to (Nint, Nfreq) boolean flag arrays
* `quals`: dict mapping antenna-pol keys to (Nint, Nfreq) float qual arrays
* `total_qual:` dict mapping polarization to (Nint, Nfreq) float total quality array

In [12]:
print(gains.keys())
print(gains[(9, 'Jxx')].shape)

[(9, 'Jxx'), (10, 'Jxx'), (20, 'Jxx'), (22, 'Jxx'), (31, 'Jxx'), (43, 'Jxx'), (53, 'Jxx'), (64, 'Jxx'), (65, 'Jxx'), (72, 'Jxx'), (80, 'Jxx'), (88, 'Jxx'), (89, 'Jxx'), (96, 'Jxx'), (97, 'Jxx'), (104, 'Jxx'), (105, 'Jxx'), (112, 'Jxx')]
(3, 1024)


Unlike DataContainers, these dictionaries have no special metadata stored internally. However, the HERACal object generates a variety of useful metadata that is stored internally.

In [13]:
print('Frequencies:', hc.freqs[0]/1e6, 'to', hc.freqs[-1]/1e6)
print('Times:', hc.times[0], 'to', hc.times[-1])
print('Antennas:', hc.ants)
print('Pols:', hc.pols)

Frequencies: 100.0 to 199.90234375
Times: 2457698.403551911 to 2457698.403800471
Antennas: [(9, 'Jxx'), (10, 'Jxx'), (20, 'Jxx'), (22, 'Jxx'), (31, 'Jxx'), (43, 'Jxx'), (53, 'Jxx'), (64, 'Jxx'), (65, 'Jxx'), (72, 'Jxx'), (80, 'Jxx'), (88, 'Jxx'), (89, 'Jxx'), (96, 'Jxx'), (97, 'Jxx'), (104, 'Jxx'), (105, 'Jxx'), (112, 'Jxx')]
Pols: ['Jxx']


To put data from dictionaries back into `HERACal` objects, again one uses the `update()` function, followed by the standard `pyuvdata.UVCal` write.

In [14]:
hc = HERACal(calfits_file)
gains, flags, quals, total_qual = hc.read()
# do stuff here
hc.update(gains=gains, flags=flags, quals=quals, total_qual=total_qual)
hc.write_calfits('new_file.calfits')
os.remove('new_file.calfits')

For now, there are no plans to support partial reading or writing for `HERACal` or any other calibration formats besides `calfits`. However, the HERAData example gives us clear mode of operation if we were to persure that option in the future.